In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
df = pd.read_csv('./data_prepared.csv')

In [4]:
from sklearn.feature_selection import VarianceThreshold

In [5]:
var_thresh = VarianceThreshold(threshold=0.1)

In [6]:
num_df = df.select_dtypes(exclude='object')

In [7]:
temp = var_thresh.fit_transform(num_df)

In [8]:
num_df.shape

(1124, 82)

In [20]:
final_df = num_df[num_df.columns[var_thresh.get_support(indices=True)]]

In [23]:
final_df.drop(['min_sal','max_sal', 'monthly_sal'], axis=1, inplace=True)

c:\python\python38\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
from sklearn.feature_selection import chi2 
from sklearn.feature_selection import f_classif 
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif 
from sklearn.feature_selection import mutual_info_regression 
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import SelectPercentile 
 
class UnivariateFeatureSelction:     
    def __init__(self, n_features, problem_type, scoring):
        """         Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.         
        :param n_features: SelectPercentile if float else SelectKBest         
        :param problem_type: classification or regression         
        :param scoring: scoring function, string         """         
        # for a given problem type, there are only         
        # a few valid scoring methods         
        # you can extend this with your own custom         
        # methods if you wish         
        if problem_type == "classification":
            valid_scoring = {                 
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }         
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }                  
        # raise exception if we do not have a valid scoring method         
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")
        
        # if n_features is int, we use selectkbest         
        # if n_features is float, we use selectpercentile         
        # please note that it is int in both cases in sklearn         
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features 
                            )         
        elif isinstance(n_features, float):             
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )         
        else:
            raise Exception("Invalid type of feature")
    
    # same fit function
    def fit(self, X, y):   
        return self.selection.fit(X, y)
    
    # same transform function     
    def transform(self, X):
        return self.selection.transform(X)
    
    # same fit_transform function
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)

In [27]:
X = num_df.drop('avg_yearly_sal', axis=1)
y = num_df['avg_yearly_sal']

In [28]:
ufs = UnivariateFeatureSelction(     
    n_features=0.1,      
    problem_type="regression",      
    scoring="f_regression" )
ufs.fit(X, y)

SelectPercentile(score_func=<function f_regression at 0x000001E59E2AC3A0>)

In [29]:
X_transformed = ufs.transform(X)

In [32]:
X_transformed.shape

(1124, 8)